#### Load HumanEval from and generate perturbations in solution code

In [1]:
import ast
import random
import pandas as pd
import difflib

from utils import Perturber

random.seed(1)

In [2]:
bugfactory = Perturber()


def perturb_humaneval(filepath):
    df = pd.read_csv(filepath)
    df['solution'] = df['prompt'] + df['canonical_solution']

    var_perturbations, expr_perturbations, func_perturbations = list(), list(), list()



    for i, row in df.iterrows():
        code = row['solution']


        var_bug = bugfactory.randomly_modify_variable(code)
        if not bugfactory.same_tree(code, var_bug):

            var_perturbations.append(var_bug)
        else:
            var_perturbations.append(None)


        expr_bug = bugfactory.randomly_modify_expression(code)
        if not bugfactory.same_tree(code, expr_bug):

            expr_perturbations.append(expr_bug)
        else:
            expr_perturbations.append(None)


        func_bug = None
        while not func_bug:
            try:
                func_bug = bugfactory.randomly_modify_functioncall(code)
            except:
                pass
        if not bugfactory.same_tree(code, func_bug):
            func_perturbations.append(func_bug)
        else:
            func_perturbations.append(None)

    return list(df['solution']), var_perturbations, expr_perturbations, func_perturbations

solution, variable_changes, expression_changes, func_changes = perturb_humaneval('data/test.csv')

#### Check out results 😄

In [3]:
print(f'Dataset size: {len(solution)}')
print(f'Successful variable perturbations: {len([1 for example in variable_changes if example is not None])}')
print(f'Successful expression perturbations: {len([1 for example in expression_changes if example is not None])}')
print(f'Successful function perturbations: {len([1 for example in function_changes if example is not None])}')

SyntaxError: f-string: expected 'else' after 'if' expression (1897632842.py, line 2)

This project is still in its weekend-hack stages, the perturbations for all possible expression types and variable types have not been implemented yet. In addition, if a particular Python function does not have more than one variable, we **cannot** randomly sample a replacement to reinsert in the code. Excited to dig deeper into ast hierarchical structure and add more possibilities soon.

Note the difference in the last line of the the two outputs below; the original solution calculates the mean absolute deviation by dividing the sum of deviations by the size of the number list, while the corrupted solution divides by the SUM of the number list.

In [19]:
print(solution[4])

from typing import List


def mean_absolute_deviation(numbers: List[float]) -> float:
    """ For a given list of input numbers, calculate Mean Absolute Deviation
    around the mean of this dataset.
    Mean Absolute Deviation is the average absolute difference between each
    element and a centerpoint (mean in this case):
    MAD = average | x - x_mean |
    >>> mean_absolute_deviation([1.0, 2.0, 3.0, 4.0])
    1.0
    """
    mean = sum(numbers) / len(numbers)
    return sum(abs(x - mean) for x in numbers) / len(numbers)



In [20]:
print(variable_changes[4])

from typing import List

def mean_absolute_deviation(numbers: List[float]) -> float:
    """ For a given list of input numbers, calculate Mean Absolute Deviation
    around the mean of this dataset.
    Mean Absolute Deviation is the average absolute difference between each
    element and a centerpoint (mean in this case):
    MAD = average | x - x_mean |
    >>> mean_absolute_deviation([1.0, 2.0, 3.0, 4.0])
    1.0
    """
    mean = sum(numbers) / len(numbers)
    return sum((abs(x - mean) for x in numbers)) / sum(numbers)
